A minimal real-world Transfer Learning pipeline.

1. Import Libraries

In [39]:
import torch

import random

import torchvision

import torch.nn as nn

import torchvision.transforms as transforms

from torchvision import models,datasets

from torch.utils.data import DataLoader, Subset

2. Load CIFAR-10 Dataset (as a stand-in for Dog vs Not-Dog)

Just extract the "dog" class and a "not-dog" class (e.g., 'airplane') to simulate a binary classification.

In [40]:
# Transform for ResNet input (resizing and normalization)

transform = transforms.Compose([

    transforms.Resize((224,224)),

    transforms.ToTensor(),

    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

In [41]:
# Load CIFAR10 dataset

train_dataset = datasets.CIFAR10(root= './data', train=True, transform=transform, download=True)

test_dataset = datasets.CIFAR10(root= './data', train=False, transform=transform, download=True)

In [42]:
# CIFAR-10 class names

class_names = train_dataset.classes

print('CIFAR-10 Classes :', class_names)

CIFAR-10 Classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [43]:
# Indices for 'dog' (class 5) and 'airplane' (class 0)

dog_idx = 5

not_dog_idx = 0

In [44]:
# Function to get subset of data for our binary task

def filter_dataset(dataset):

    indices = [i for i ,(_, label) in enumerate(dataset) if label in (dog_idx, not_dog_idx)]

    filtered = Subset(dataset, indices)

    return filtered

In [45]:
train_data = filter_dataset(train_dataset)

test_data = filter_dataset(test_dataset)


train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

3. Load Pretrained Model (ResNet18)

In [46]:
# Load pre-trained model

model = models.resnet18(pretrained = True)

# Freeze all layers

for param in model.parameters():

    param.requires_grad = False


# Replace the final fully connected layer for 2 classes

num_features = model.fc.in_features

model.fc = nn.Linear(num_features, 2)  # Dog vs Not-Dog

4. Define Training Essentials

In [47]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

model = model.to(device)


criterion  = nn. CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

5. Training Loop (Basic)

In [48]:
# Few epochs for demo

for epoch in range(3):

    model.train()

    running_loss = 0

    for images, labels in train_loader:

        images, labels = images.to(device), labels.to(device)

        labels = (labels == dog_idx).long() # Make it 0 (not dog) or 1 (dog)



        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()


        running_loss += loss.item()

    print(f'Epoch  {epoch + 1},Loss : {running_loss / len(train_loader):.4f}')


Epoch  1,Loss : 0.1399
Epoch  2,Loss : 0.0872
Epoch  3,Loss : 0.0741


6. Evaluation (Quick Accuracy)

In [49]:
model.eval()

correct, total = 0, 0



with torch.no_grad():

    for images, labels in test_loader:

        images, labels = images.to(device), labels.to(device)

        labels = (labels == dog_idx).long()


        outputs = model(images)

        _, preds = torch.max(outputs, 1)

        correct += (preds == labels).sum().item()

        total = labels.size(0)

print(f'Test Accuracy : {100 * correct / total :.2f}%')

Test Accuracy : 12306.25%


What We Just Did:

Used CIFAR-10 as dummy binary data
Loaded pretrained ResNet18
Froze its weights, replaced its head
Trained for Dog vs Not-Dog with high accuracy in very few epochs